# Memory
- Invoke Openai model
- Create simple chain and invoke
- Add chat history using: **ChatMessageHistory**
- Add chat history using: **RunnableWithMessageHistory**
- Langchain OpenAI and use ConversationChain
- LangChain - ConversationBufferMemory



In [1]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.2 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage  # new message class in langchain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# iNVOKE OPENAPI KEY
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY


## Invoke Openai model

In [6]:
# INVOKE OPENAPI MODEL
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

#OPENAI ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "you are a helpful assistant. Answer all the question to the best of your ability."
    ),
    MessagesPlaceholder(variable_name = "messages"),  # User Q saved in messages parameter
]
)

## Create simple chain and invoke

In [ ]:
chain =  prompt | llm

resposne = chain.invoke(
    {
        # This messsages parameter should be same as above prompt parameter
        # Here conversational message sample kept inside messagees
       "messages": [

                   HumanMessage(
                       content= "translate this given sentence from english to french: I LOVE AI."  # Human Q
                   ),
                   AIMessage(content= "J'adore la AI."),      # AI Response

                   HumanMessage(content= "what did you just say?"),  # Human Q

       ] ,

    }
)

resposne.content

'I said "I love AI" in French.'

## Add chat history using: ChatMessageHistory

In [11]:
from langchain.memory import ChatMessageHistory

In [ ]:
demo_chat_history=ChatMessageHistory()
demo_chat_history.add_user_message("translate this given sentence from english to french: I LOVE AI.")
demo_chat_history.add_ai_message("J'adore la AI.")
demo_chat_history.messages  #displaay message history

[HumanMessage(content='translate this given sentence from english to french: I LOVE AI.'),
 AIMessage(content="J'adore la AI.")]

### ChatMessageHistory - Example 1

In [ ]:
demo_chat_history=ChatMessageHistory()
input1="who is a prime minister of india?"
demo_chat_history.add_user_message(input1)
demo_chat_history.messages

[HumanMessage(content='who is a prime minister of india?')]

In [ ]:
resposne=chain.invoke(
    {
        "messages": demo_chat_history.messages
    }
)

resposne.content

'As of my last update, the Prime Minister of India is Narendra Modi. He has been in office since May 2014.'

### ChatMessageHistory - Example 2

In [ ]:
demo_chat_history.add_ai_message(resposne)
input2="what did i ask to you just now?"
demo_chat_history.add_user_message(input2)
demo_chat_history.messages

[HumanMessage(content='who is a prime minister of india?'),
 AIMessage(content='As of my last update, the Prime Minister of India is Narendra Modi. He has been in office since May 2014.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 36, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_3d37c73133', 'finish_reason': 'stop', 'logprobs': None}, id='run-b2a844d2-f60c-48fa-83df-ebd2f7381a48-0', usage_metadata={'input_tokens': 36, 'output_tokens': 26, 'total_tokens': 62}),
 HumanMessage(content='what did i ask to you just now?')]

In [ ]:
response2 = chain.invoke(
    {
        "messages": demo_chat_history.messages
    }
)
response2.content

'You asked about the current Prime Minister of India.'

## Add chat history using: RunnableWithMessageHistory

### ChatMessageHistory -  with prompt 1

In [ ]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain2 = prompt | llm

### ChatMessageHistory -  with prompt 2

In [ ]:
prompt3 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt3 | llm


In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

demo_chat_history2 = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
                              chain,
                              lambda session_id: demo_chat_history2,
                              input_message_key = "input",
                              history_message_key ="chat_history",
                              )

chain_with_message_history.invoke(
    {"input": "Translate this sentence from English to French: I love programming."},
    {"configurable": {"session_id": "unused"}}
)

AIMessage(content='The translation of "I love programming" from English to French is "J\'adore la programmation."', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 45, 'total_tokens': 67}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_3d37c73133', 'finish_reason': 'stop', 'logprobs': None}, id='run-d248d27c-313a-4713-bc03-c3c1883b7e3a-0', usage_metadata={'input_tokens': 45, 'output_tokens': 22, 'total_tokens': 67})

In [ ]:
chain_with_message_history.invoke(
    {"input": "What did I just ask you?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='You just asked me to translate the sentence "I love programming" from English to French.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 210, 'total_tokens': 228}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_3d37c73133', 'finish_reason': 'stop', 'logprobs': None}, id='run-6bcf552b-19c2-4414-989d-1ce291d0c664-0', usage_metadata={'input_tokens': 210, 'output_tokens': 18, 'total_tokens': 228})

## Langchain OpenAI and use ConversationChain

In [ ]:
from langchain import OpenAI

In [ ]:
'''llm = OpenAI(
	temperature=0,
 model_name="gpt-3.5-turbo-1106"
)
 '''

/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:1072: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
from langchain.chains import ConversationChain
conversation = ConversationChain(llm=llm)
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
question1="i am interested in exploring the potential of large language model with external knowledge"
question2="i would like to analysis all the different possibility, what can you think of?"
question3="which data source tyoe could be used to give context to the model?"

## LangChain - ConversationBufferMemory

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory= ConversationBufferMemory()

)

conversation_buf("good morning my AI")

{'input': 'good morning my AI',
 'history': '',
 'response': 'Good morning! How are you today? Did you sleep well? The weather outside is currently 65 degrees Fahrenheit with a 30% chance of rain. How can I help you?'}

### function to get total toke count

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain,query):
  with get_openai_callback() as cb:
    result= chain.run(query)
    print(f"totla no of token is {cb.total_tokens}")
  return result

In [ ]:
question1="i am interested in exploring the potential of large language model with external knowledge"

count_tokens(conversation_buf,question1)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


totla no of token is 233


"That's a fascinating topic! Large language models with external knowledge have shown great potential in improving natural language understanding and generation. By incorporating external knowledge sources such as encyclopedias, databases, and websites, these models can provide more accurate and informative responses to user queries. They can also better understand the context of a conversation and provide more contextually relevant information. There are various research papers and articles available that delve into the details and potential applications of these models. Is there a specific aspect of this topic that you would like to explore further?"

In [ ]:
conversation_buf(question1)

{'input': 'i am interested in exploring the potential of large language model with external knowledge',
 'history': "Human: good morning my AI\nAI: Good morning! How are you today? Did you sleep well? The weather outside is currently 65 degrees Fahrenheit with a 30% chance of rain. How can I help you?\nHuman: i am interested in exploring the potential of large language model with external knowledge\nAI: That's a fascinating topic! Large language models with external knowledge have shown great potential in improving natural language understanding and generation. By incorporating external knowledge sources such as encyclopedias, databases, and websites, these models can provide more accurate and informative responses to user queries. They can also better understand the context of a conversation and provide more contextually relevant information. There are various research papers and articles available that delve into the details and potential applications of these models. Is there a spec

In [ ]:
question2="i would like to analysis all the different possibility, what can you think of?"
conversation_buf(question2)

{'input': 'i would like to analysis all the different possibility, what can you think of?',
 'history': "Human: good morning my AI\nAI: Good morning! How are you today? Did you sleep well? The weather outside is currently 65 degrees Fahrenheit with a 30% chance of rain. How can I help you?\nHuman: i am interested in exploring the potential of large language model with external knowledge\nAI: That's a fascinating topic! Large language models with external knowledge have shown great potential in improving natural language understanding and generation. By incorporating external knowledge sources such as encyclopedias, databases, and websites, these models can provide more accurate and informative responses to user queries. They can also better understand the context of a conversation and provide more contextually relevant information. There are various research papers and articles available that delve into the details and potential applications of these models. Is there a specific aspect 

In [ ]:
count_tokens(conversation_buf,question2)

totla no of token is 673


'Some potential areas of exploration could include the impact of different types of external knowledge sources on the performance of large language models, the development of more efficient algorithms for integrating external knowledge, and the ethical considerations of using external knowledge in language generation. Additionally, you could also explore the potential applications of large language models with external knowledge in various industries such as healthcare, finance, and customer service. There are also opportunities to investigate the potential limitations and challenges of these models, such as biases in the external knowledge sources and the potential for misinformation. These are just a few possibilities, and there are many more aspects to consider in exploring the potential of large language models with external knowledge.'

Some potential areas of exploration could include the impact of different types of external knowledge sources on the performance of large language models, the development of more efficient algorithms for integrating external knowledge, and the ethical considerations of using external knowledge in language generation. Additionally, you could also explore the potential applications of large language models with external knowledge in various industries such as healthcare, finance, and customer service. There are also opportunities to investigate the potential limitations and challenges of these models, such as biases in the external knowledge sources and the potential for misinformation. These are just a few possibilities, and there are many more aspects to consider in exploring the potential of large language models with external knowledge.

In [ ]:
question3

'which data source tyoe could be used to give context to the model?'

In [ ]:
conversation_buf(question3)['response']

'Different types of data sources that could be used to give context to the model include structured databases, unstructured text from sources like news articles or books, knowledge graphs, and domain-specific knowledge bases. These data sources can provide the model with a rich understanding of various topics and domains, allowing it to generate more accurate and contextually relevant responses. Additionally, real-time data from sources such as social media or online forums can also be used to provide up-to-date context for the model. Each type of data source has its own strengths and limitations, and exploring the potential of each could provide valuable insights into enhancing the capabilities of large language models with external knowledge.'

# Assignment
- take atleast 5 data source(1 ppt, 1 weblodaer, 1 pdf, 1txt, 1csv)
- maintain one knowledge base(mongodb, astradb , pineconde, weviate)
- user will ask you have to provide answer based on the asked question using this know;ledge base
- handle the comman question as well like hi hello how are you good evening good morning etc.
- you have to mention the complete memory of the conversation here the threshold is 10
- then create a UI for you bot

https://www.youtube.com/@sunnysavita10/videos
